In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
path_data = os.getenv("path_to_data")
path_master = os.getenv("path_to_master")

In [3]:
df_wetter = pd.read_csv(f"{path_data}/clean_wetter_komplett.csv", header = 0)

In [4]:
df_wetter.set_index("Unnamed: 0", inplace = True)

In [5]:
df_messungen = pd.read_csv(f"{path_master}/messungen_clean_2024.csv", header = 0)

In [6]:
df_messungen.set_index("DATUM", inplace = True)

In [7]:
df_komplett = pd.merge(df_wetter, df_messungen, how = "left", left_index=True, right_index=True)

In [8]:
df_komplett.index.rename("zeit", inplace=True)
df_komplett.index = pd.to_datetime(df_komplett.index)

In [9]:
zeit_max = df_komplett[pd.notna(df_komplett["ecoli"])].index.max()
zeit_min = df_komplett[pd.notna(df_komplett["ecoli"])].index.min()

In [10]:
df_komplett = df_komplett[df_komplett.index > zeit_min- pd.Timedelta("7d")]
df_komplett = df_komplett[df_komplett.index < zeit_max + pd.Timedelta("7d")]

In [11]:
df_wendlingen= pd.read_csv(f"{path_data}/wendlingen_messwerte.csv", delimiter = ";", parse_dates = True, index_col = 0)

In [12]:
df_wendlingen = df_wendlingen.pivot(index = "Datum", columns = "Parameter", values = "Tagesmittelwert")

In [13]:
df_wendlingen.index = pd.to_datetime(df_wendlingen.index, format = "%d.%m.%Y")

In [14]:
df_wendlingen = df_wendlingen.resample("1h").first().ffill()

In [15]:
for col in df_wendlingen:
    df_wendlingen[col] = df_wendlingen[col].str.replace(",", ".")
    try:
        df_wendlingen[col] = pd.to_numeric(df_wendlingen[col])
    except:
        pass

In [16]:
df_komplett = df_komplett.merge(df_wendlingen, left_index=True, right_index=True)

In [17]:
df_komplett.reset_index(inplace = True)

In [18]:
df_komplett.columns = [col.strip() for col in df_komplett.columns]

In [19]:
df_komplett.to_csv(f"{path_master}/masterdata_2024.csv")